# CLASSIFICATION with LSTM Network in Keras

In [109]:
# Import the necessary libraries, modules
import pandas as pd # Pandas library for reading '.csv' files as dataframes
import numpy as np # Numpy library for creating and modifying arrays.
from keras.layers import Dense, SimpleRNN, GRU, LSTM, Embedding ,Dropout,BatchNormalization# Import layers from Keras
from keras.models import Sequential

In [2]:
import os
os.chdir('C:\\Users\\atul\\Desktop')

### Reading the data

In [3]:
raw_data = pd.read_csv('AuthorsDataset.csv', encoding='latin-1')


print(raw_data.shape)
print(raw_data.columns)
print(raw_data)

(750, 2)
Index(['Text', 'Author'], dtype='object')
                                                  Text     Author
0    I give Pirrip as my father's family name, on t...    Charles
1    Ours was the marsh country, down by the river,...    Charles
2    You bring me, to-morrow morning early, that fi...    Charles
3    The marshes were just a long black horizontal ...    Charles
4    My sister, Mrs. Joe, with black hair and eyes,...    Charles
5    My sister had a trenchant way of cutting our b...    Charles
6    The effort of resolution necessary to the achi...    Charles
7    Some medical beast had revived Tar-water in th...    Charles
8    Conscience is a dreadful thing when it accuses...    Charles
9    As soon as the great black velvet pall outside...    Charles
10   The mist was heavier yet when I got out upon t...    Charles
11   All this time, I was getting on towards the ri...    Charles
12   Why, see now! said he. When a man's alone on t...    Charles
13   I indicated in what 

In [4]:
from sklearn.utils import shuffle

data= shuffle(raw_data)

print(data)

                                                  Text     Author
352  No doubt for first offenders, and for all offe...    HGWells
7    Some medical beast had revived Tar-water in th...    Charles
635  This was all wretched for a stranger like myse...  Stevenson
56   When I got up to my little room and said my pr...    Charles
60   I crossed the staircase landing, and entered t...    Charles
2    You bring me, to-morrow morning early, that fi...    Charles
191  By this time we had come to the house, where I...    Charles
594  The second day I crossed the island to all sid...  Stevenson
731  Why, so it would appear, says he, filling his ...  Stevenson
271  Individual liberty in a community is not, as m...    HGWells
113  My sister was never left alone now; but Joe mo...    Charles
510  It was drawing on to sundown when I met a stou...  Stevenson
254  There must always be a certain effect of hardn...    HGWells
631  Never a word they spoke as they pulled ashore,...  Stevenson
73   Meanw

In [5]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(data, test_size = 0.2) 
print(train.shape)
print(test.shape)

(600, 2)
(150, 2)


In [6]:
train.columns

Index(['Text', 'Author'], dtype='object')

### Converting unstructured text to structured numeric form
This includes:
1. Tokenizing
2. Converting sequence of words to sequence of word indeces
3. Converting varing length sequences to fixed length sequences through padding

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train.Text)

train_x = tokenizer.texts_to_sequences(train.Text)
train_x = pad_sequences(train_x, maxlen=100)
test_x = tokenizer.texts_to_sequences(test.Text)
test_x = pad_sequences(test_x, maxlen=100)

train_x.shape, test_x.shape

((600, 100), (150, 100))

In [12]:
print(train_x[2])

[ 101  622    9  408 2806 5610    6   14 5611 3701    4  479    5 2227
   15   23    2   25    7  163  155   15   14 5612    2 5613  883   22
    1 5614  255   16    8    9   10 1828    3    1  182 5615    3   14
  754   94  104    7   10 2228 5616    2  530   17 1358    1  391  378
  312 2229   21   23    2 3702   19  183    2  589  180 2230  272  180
 2230   11   49    9   35    5  113  667  531    8    7 2807 5617    5
  256  590 1208    7   71   80 3703   40    4   27    1 1556    4  462
   14 2808]


In [42]:
from keras.utils import to_categorical # This convers the labels to one-hot vectors(Dummies)
unique_labels = list(data.Author.unique())
train_y = np.array([unique_labels.index(i) for i in train.Author])
train_y = to_categorical(train_y)
test_y_ = np.array([unique_labels.index(i) for i in test.Author])
test_y = to_categorical(test_y_)

In [43]:
import keras.backend as K # This 'K' can be used to create user defined functions in keras

# Define a custom function in keras to compute recall.
# Arguments:
# y_true - Actual labels
# y_pred - Predicted labels
def recall(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    PP = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = TP / (PP + K.epsilon())
    return recall

### Building and training an LSTM model

In [53]:
# Building an LSTM model
look_back=4 ##here i am giving to 4 words at a time stamp
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=100, 
                    output_dim=50)) # Add an embedding layer which represents each unique token as a vector
model.add(LSTM(7,input_shape=(1, look_back), return_sequences=False)) # Add an LSTM layer
model.add(Dense(3, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes for 3 classes.

In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 50)           581550    
_________________________________________________________________
lstm_9 (LSTM)                (None, 7)                 1624      
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 24        
Total params: 583,198
Trainable params: 583,198
Non-trainable params: 0
_________________________________________________________________


In [55]:
print(train_y.shape)
print(test_y.shape)
print(train_x.shape)
print(test_x.shape)

(600, 3)
(150, 3)
(600, 100)
(150, 100)


In [56]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)

Train on 450 samples, validate on 150 samples
Epoch 1/20
450/450 [==============================] - 4s 8ms/step - loss: 1.0969 - acc: 0.3933 - val_loss: 1.0948 - val_acc: 0.4000
Epoch 2/20
450/450 [==============================] - 1s 2ms/step - loss: 1.0817 - acc: 0.6333 - val_loss: 1.0901 - val_acc: 0.4267
Epoch 3/20
450/450 [==============================] - 1s 2ms/step - loss: 1.0617 - acc: 0.8022 - val_loss: 1.0829 - val_acc: 0.4933
Epoch 4/20
450/450 [==============================] - 1s 2ms/step - loss: 1.0298 - acc: 0.9022 - val_loss: 1.0704 - val_acc: 0.5667
Epoch 5/20
450/450 [==============================] - 1s 2ms/step - loss: 0.9744 - acc: 0.9422 - val_loss: 1.0474 - val_acc: 0.6333
Epoch 6/20
450/450 [==============================] - 1s 2ms/step - loss: 0.8767 - acc: 0.9356 - val_loss: 0.9885 - val_acc: 0.6133
Epoch 7/20
450/450 [==============================] - 1s 2ms/step - loss: 0.7192 - acc: 0.9178 - val_loss: 0.8611 - val_acc: 0.6400
Epoch 8/20
450/450 [==========

### Prediction and evaluation on test data
1. Check the network output on test data. What do these values represent?
2. Predict the class labels on test data
2. Evaluate the model on test data

Hint: Check model.predict, model.predict_classes, model.evaluate in keras

In [70]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',recall])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)

Train on 450 samples, validate on 150 samples
Epoch 1/20
450/450 [==============================] - 3s 7ms/step - loss: 0.0147 - acc: 0.9978 - recall: 0.9978 - val_loss: 0.7791 - val_acc: 0.7733 - val_recall: 0.7733
Epoch 2/20
450/450 [==============================] - 1s 3ms/step - loss: 0.0112 - acc: 1.0000 - recall: 1.0000 - val_loss: 0.9063 - val_acc: 0.7267 - val_recall: 0.7200
Epoch 3/20
450/450 [==============================] - 1s 2ms/step - loss: 0.0099 - acc: 1.0000 - recall: 1.0000 - val_loss: 0.9393 - val_acc: 0.7200 - val_recall: 0.7067
Epoch 4/20
450/450 [==============================] - 1s 2ms/step - loss: 0.0087 - acc: 1.0000 - recall: 1.0000 - val_loss: 0.9521 - val_acc: 0.7200 - val_recall: 0.7133
Epoch 5/20
450/450 [==============================] - 1s 3ms/step - loss: 0.0077 - acc: 1.0000 - recall: 1.0000 - val_loss: 0.9986 - val_acc: 0.7267 - val_recall: 0.7200
Epoch 6/20
450/450 [==============================] - 1s 2ms/step - loss: 0.0069 - acc: 1.0000 - recall:

In [68]:
test_prob = model.predict(test_x)
test_prob[5]

array([0.07045849, 0.8476717 , 0.08186986], dtype=float32)

In [63]:
test_prob = model.predict(test_x)
test_prob[5]
test_classes = model.predict_classes(test_x)
test_classes.shape
print(test_classes)

[2 1 1 0 2 1 2 2 1 1 1 1 2 0 0 0 1 1 1 1 0 0 0 2 2 1 0 0 1 1 2 1 1 1 2 0 0
 0 1 1 0 2 1 1 1 1 0 1 1 0 2 2 2 1 2 2 0 2 1 0 1 0 1 1 1 0 1 1 2 0 1 2 0 1
 2 2 2 1 0 2 1 1 0 0 1 1 0 0 2 1 2 2 2 2 0 1 0 1 2 1 1 2 1 1 1 1 1 1 1 0 1
 1 1 2 1 2 1 0 0 0 2 0 1 1 2 2 2 1 2 1 1 1 1 1 2 0 2 1 0 1 2 1 1 1 2 2 1 0
 2 1]


In [65]:
import keras.backend as K # This 'K' can be used to create user defined functions in keras

# Define a custom function in keras to compute recall.
# Arguments:
# y_true - Actual labels
# y_pred - Predicted labels
def recall(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    PP = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = TP / (PP + K.epsilon())
    return recall

<function __main__.recall>

# Now tuning the privious model

### Understanding an intermediate layer in keras


In [71]:
# Building an LSTM model
look_back=15##here i am giving to 15 words at a time stamp
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=100, 
                    output_dim=100)) # Add an embedding layer which represents each unique token as a vector
model.add(LSTM(15,input_shape=(1, look_back), return_sequences=False)) # Add an LSTM layer
model.add(Dense(3, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes for 3 classes.

In [72]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',recall])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)

Train on 450 samples, validate on 150 samples
Epoch 1/20
450/450 [==============================] - 4s 8ms/step - loss: 1.0951 - acc: 0.3911 - recall: 0.0000e+00 - val_loss: 1.0902 - val_acc: 0.4733 - val_recall: 0.0000e+00
Epoch 2/20
450/450 [==============================] - 1s 3ms/step - loss: 1.0527 - acc: 0.7422 - recall: 0.0000e+00 - val_loss: 1.0715 - val_acc: 0.4600 - val_recall: 0.0000e+00
Epoch 3/20
450/450 [==============================] - 1s 3ms/step - loss: 0.9636 - acc: 0.7667 - recall: 0.0022 - val_loss: 1.0013 - val_acc: 0.5333 - val_recall: 0.0067
Epoch 4/20
450/450 [==============================] - 1s 3ms/step - loss: 0.7455 - acc: 0.6911 - recall: 0.4756 - val_loss: 0.8435 - val_acc: 0.5867 - val_recall: 0.4400
Epoch 5/20
450/450 [==============================] - 1s 3ms/step - loss: 0.5394 - acc: 0.7356 - recall: 0.6689 - val_loss: 0.6825 - val_acc: 0.6600 - val_recall: 0.5733
Epoch 6/20
450/450 [==============================] - 1s 3ms/step - loss: 0.3697 - acc: 

In [ ]:
##above result is the biasing case then need more tuning

In [75]:
# Building an LSTM model
look_back=715##here i am giving to 4 words at a time stamp
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=100, 
                    output_dim=100)) # Add an embedding layer which represents each unique token as a vector
model.add(LSTM(8,input_shape=(1, look_back), return_sequences=False)) # Add an LSTM layer
model.add(Dense(3, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes 

In [76]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',recall])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)

Train on 450 samples, validate on 150 samples
Epoch 1/20
450/450 [==============================] - 4s 9ms/step - loss: 1.0965 - acc: 0.3978 - recall: 0.0000e+00 - val_loss: 1.0910 - val_acc: 0.4667 - val_recall: 0.0000e+00
Epoch 2/20
450/450 [==============================] - 1s 3ms/step - loss: 1.0704 - acc: 0.6822 - recall: 0.0000e+00 - val_loss: 1.0814 - val_acc: 0.5467 - val_recall: 0.0000e+00
Epoch 3/20
450/450 [==============================] - 1s 3ms/step - loss: 1.0299 - acc: 0.8022 - recall: 0.0000e+00 - val_loss: 1.0591 - val_acc: 0.5733 - val_recall: 0.0000e+00
Epoch 4/20
450/450 [==============================] - 1s 3ms/step - loss: 0.9474 - acc: 0.9222 - recall: 0.0000e+00 - val_loss: 1.0053 - val_acc: 0.6200 - val_recall: 0.0000e+00
Epoch 5/20
450/450 [==============================] - 1s 3ms/step - loss: 0.7935 - acc: 0.9600 - recall: 0.1756 - val_loss: 0.8916 - val_acc: 0.6733 - val_recall: 0.1133
Epoch 6/20
450/450 [==============================] - 2s 4ms/step - loss

In [ ]:
##Now I am using drop outs for over fitting

In [80]:
# Building an LSTM model
look_back=7##here i am giving to 4 words at a time stamp
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=100, 
                    output_dim=100)) # Add an embedding layer which represents each unique token as a vector
model.add(Dropout(0.5))
model.add(LSTM(15,input_shape=(1, look_back), return_sequences=False)) # Add an LSTM layer

model.add(Dense(3, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes 

In [81]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',recall])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)

Train on 450 samples, validate on 150 samples
Epoch 1/20
450/450 [==============================] - 4s 9ms/step - loss: 1.0973 - acc: 0.3489 - recall: 0.0000e+00 - val_loss: 1.0905 - val_acc: 0.5000 - val_recall: 0.0000e+00
Epoch 2/20
450/450 [==============================] - 1s 3ms/step - loss: 1.0746 - acc: 0.6911 - recall: 0.0000e+00 - val_loss: 1.0802 - val_acc: 0.5933 - val_recall: 0.0000e+00
Epoch 3/20
450/450 [==============================] - 1s 3ms/step - loss: 1.0294 - acc: 0.8644 - recall: 0.0000e+00 - val_loss: 1.0382 - val_acc: 0.6467 - val_recall: 0.0000e+00
Epoch 4/20
450/450 [==============================] - 2s 4ms/step - loss: 0.8582 - acc: 0.8911 - recall: 0.1911 - val_loss: 0.8690 - val_acc: 0.7067 - val_recall: 0.2200
Epoch 5/20
450/450 [==============================] - 1s 3ms/step - loss: 0.6455 - acc: 0.9422 - recall: 0.3467 - val_loss: 0.9279 - val_acc: 0.6467 - val_recall: 0.1400
Epoch 6/20
450/450 [==============================] - 1s 3ms/step - loss: 0.5619

In [98]:
# Building an LSTM model
look_back=7##here i am giving to 4 words at a time stamp
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=100, 
                    output_dim=150)) # Add an embedding layer which represents each unique token as a vector
model.add(Dropout(0.4))
model.add(LSTM(12,input_shape=(1, look_back), return_sequences=False)) # Add an LSTM layer


model.add(Dense(3, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes 

In [97]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',recall])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)

Train on 450 samples, validate on 150 samples
Epoch 1/20
450/450 [==============================] - 5s 12ms/step - loss: 1.0940 - acc: 0.3956 - recall: 0.0000e+00 - val_loss: 1.0869 - val_acc: 0.4600 - val_recall: 0.0000e+00
Epoch 2/20
450/450 [==============================] - 2s 4ms/step - loss: 1.0562 - acc: 0.6444 - recall: 0.0000e+00 - val_loss: 1.0692 - val_acc: 0.4600 - val_recall: 0.0000e+00
Epoch 3/20
450/450 [==============================] - 2s 4ms/step - loss: 0.9897 - acc: 0.7000 - recall: 0.0000e+00 - val_loss: 1.0230 - val_acc: 0.4800 - val_recall: 0.0000e+00
Epoch 4/20
450/450 [==============================] - 2s 4ms/step - loss: 0.8170 - acc: 0.7022 - recall: 0.3222 - val_loss: 0.8864 - val_acc: 0.6000 - val_recall: 0.3800
Epoch 5/20
450/450 [==============================] - 2s 4ms/step - loss: 0.5993 - acc: 0.7933 - recall: 0.6467 - val_loss: 0.7508 - val_acc: 0.6800 - val_recall: 0.5067
Epoch 6/20
450/450 [==============================] - 2s 4ms/step - loss: 0.487

In [99]:
test_prob = model.predict(test_x)
test_prob[5]
test_classes = model.predict_classes(test_x)
test_classes.shape
print(test_classes)

[1 1 2 0 2 2 1 0 2 0 2 0 2 1 0 0 2 0 2 1 1 0 0 0 1 0 2 0 0 1 2 2 2 1 1 1 2
 2 2 2 0 2 1 1 2 0 2 0 0 0 0 2 1 1 0 2 2 1 0 2 1 0 2 0 1 0 0 1 0 0 1 2 0 2
 0 0 0 0 0 2 2 1 0 0 1 0 0 2 2 0 2 1 1 0 1 0 1 2 2 1 0 0 2 0 2 0 2 0 2 2 2
 2 2 2 2 2 2 2 2 2 1 0 2 1 0 2 0 1 2 0 1 2 0 0 1 0 2 1 0 2 2 2 0 0 2 0 1 1
 0 2]


# now using Optimizer='rmsprop'

In [101]:
look_back=7##here i am giving to 4 words at a time stamp
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=100, 
                    output_dim=150)) # Add an embedding layer which represents each unique token as a vector
model.add(Dropout(0.4))
model.add(LSTM(12,input_shape=(1, look_back), return_sequences=False)) # Add an LSTM layer


model.add(Dense(3, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes


In [102]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy',recall])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)

Train on 450 samples, validate on 150 samples
Epoch 1/20
450/450 [==============================] - 6s 12ms/step - loss: 1.0929 - acc: 0.4067 - recall: 0.0000e+00 - val_loss: 1.0899 - val_acc: 0.4333 - val_recall: 0.0000e+00
Epoch 2/20
450/450 [==============================] - 2s 4ms/step - loss: 1.0541 - acc: 0.6978 - recall: 0.0000e+00 - val_loss: 1.0696 - val_acc: 0.5133 - val_recall: 0.0000e+00
Epoch 3/20
450/450 [==============================] - 2s 5ms/step - loss: 0.9779 - acc: 0.7933 - recall: 0.0044 - val_loss: 1.0125 - val_acc: 0.5200 - val_recall: 0.0067
Epoch 4/20
450/450 [==============================] - 2s 4ms/step - loss: 0.7671 - acc: 0.8067 - recall: 0.4533 - val_loss: 0.8172 - val_acc: 0.6667 - val_recall: 0.4933
Epoch 5/20
450/450 [==============================] - 2s 4ms/step - loss: 0.5747 - acc: 0.8778 - recall: 0.6622 - val_loss: 0.6852 - val_acc: 0.7800 - val_recall: 0.5933
Epoch 6/20
450/450 [==============================] - 2s 4ms/step - loss: 0.4368 - acc:

In [104]:
look_back=7##here i am giving to 4 words at a time stamp
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=100, 
                    output_dim=150)) # Add an embedding layer which represents each unique token as a vector
model.add(Dropout(0.4))
model.add(LSTM(12,input_shape=(1, look_back), return_sequences=False)) # Add an LSTM layer


model.add(Dense(3, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes

In [110]:
from keras import optimizers

###################ADAM hyper tune

In [115]:
##Hypertuning The Adam
adam = optimizers.Adam(lr=0.01, decay=1e-6, beta_1=.8 ,beta_2=.85)


In [116]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',recall])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)


Train on 450 samples, validate on 150 samples
Epoch 1/20
450/450 [==============================] - 7s 15ms/step - loss: 0.0428 - acc: 0.9978 - recall: 0.9978 - val_loss: 0.6059 - val_acc: 0.7400 - val_recall: 0.7400
Epoch 2/20
450/450 [==============================] - 2s 4ms/step - loss: 0.0334 - acc: 1.0000 - recall: 1.0000 - val_loss: 0.6511 - val_acc: 0.7333 - val_recall: 0.7333
Epoch 3/20
450/450 [==============================] - 2s 4ms/step - loss: 0.0247 - acc: 1.0000 - recall: 1.0000 - val_loss: 0.6396 - val_acc: 0.7533 - val_recall: 0.7533
Epoch 4/20
450/450 [==============================] - 2s 4ms/step - loss: 0.0335 - acc: 0.9956 - recall: 0.9956 - val_loss: 0.5885 - val_acc: 0.7800 - val_recall: 0.7800
Epoch 5/20
450/450 [==============================] - 2s 4ms/step - loss: 0.0462 - acc: 0.9889 - recall: 0.9889 - val_loss: 0.6471 - val_acc: 0.7667 - val_recall: 0.7600
Epoch 6/20
450/450 [==============================] - 2s 4ms/step - loss: 0.0155 - acc: 1.0000 - recall

In [ ]:
##now I am using normalization

In [92]:
# Building an LSTM model
look_back=7##here i am giving to 4 words at a time stamp

model = Sequential() # Call Sequential to initialize a network

model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=100, 
                    output_dim=100)) # Add an embedding layer which represents each unique token as a vector
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(LSTM(15,input_shape=(1, look_back), return_sequences=False)) # Add an LSTM layer

model.add(Dense(3, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes 

In [93]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',recall])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)


Train on 450 samples, validate on 150 samples
Epoch 1/20
450/450 [==============================] - 5s 12ms/step - loss: 1.1099 - acc: 0.3644 - recall: 0.0267 - val_loss: 1.0568 - val_acc: 0.4067 - val_recall: 0.0200
Epoch 2/20
450/450 [==============================] - 2s 4ms/step - loss: 0.9667 - acc: 0.5844 - recall: 0.1289 - val_loss: 1.0142 - val_acc: 0.4667 - val_recall: 0.0800
Epoch 3/20
450/450 [==============================] - 2s 4ms/step - loss: 0.8539 - acc: 0.7022 - recall: 0.2889 - val_loss: 0.9821 - val_acc: 0.4933 - val_recall: 0.1667
Epoch 4/20
450/450 [==============================] - 2s 4ms/step - loss: 0.7360 - acc: 0.8200 - recall: 0.4733 - val_loss: 0.9560 - val_acc: 0.5333 - val_recall: 0.2333
Epoch 5/20
450/450 [==============================] - 2s 4ms/step - loss: 0.5966 - acc: 0.8867 - recall: 0.7089 - val_loss: 0.9293 - val_acc: 0.5267 - val_recall: 0.2667
Epoch 6/20
450/450 [==============================] - 2s 4ms/step - loss: 0.4726 - acc: 0.9333 - recall

## You can see that when I use normalzation accuracy dicreases

In [117]:
look_back=7##here i am giving to 4 words at a time stamp
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=100, 
                    output_dim=150)) # Add an embedding layer which represents each unique token as a vector
model.add(Dropout(0.4))
model.add(LSTM(12,input_shape=(1, look_back), return_sequences=False)) # Add an LSTM layer


model.add(Dense(3, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy',recall])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)

Train on 450 samples, validate on 150 samples
Epoch 1/20
450/450 [==============================] - 6s 13ms/step - loss: 1.0952 - acc: 0.3778 - recall: 0.0000e+00 - val_loss: 1.0877 - val_acc: 0.5067 - val_recall: 0.0000e+00
Epoch 2/20
450/450 [==============================] - 2s 4ms/step - loss: 1.0621 - acc: 0.6733 - recall: 0.0000e+00 - val_loss: 1.0757 - val_acc: 0.5600 - val_recall: 0.0000e+00
Epoch 3/20
450/450 [==============================] - 2s 5ms/step - loss: 1.0125 - acc: 0.7844 - recall: 0.0000e+00 - val_loss: 1.0553 - val_acc: 0.5200 - val_recall: 0.0000e+00
Epoch 4/20
450/450 [==============================] - 2s 5ms/step - loss: 0.9177 - acc: 0.7733 - recall: 0.1111 - val_loss: 1.0293 - val_acc: 0.4467 - val_recall: 0.1533
Epoch 5/20
450/450 [==============================] - 2s 4ms/step - loss: 0.7306 - acc: 0.8133 - recall: 0.3644 - val_loss: 1.0367 - val_acc: 0.5267 - val_recall: 0.3200
Epoch 6/20
450/450 [==============================] - 2s 5ms/step - loss: 0.560

# Transfer Learning

In [118]:
model.layers

In [125]:
import keras.backend as K 
# Create a user defined function in keras, where we mention the input and output
# This function returns a list
eo = K.function([model.layers[0].input],
                  [model.layers[0].output])

out = eo([train_x[0:5]]) 
print(type(out))
print(len(out))
print(out[0].shape)

<class 'list'>
1
(5, 100, 150)


In [124]:
out[0]

array([[[-0.00406499, -0.03752612,  0.02748853, ...,  0.03032188,
         -0.01770221, -0.0141458 ],
        [-0.01464888, -0.00491512,  0.07457757, ...,  0.04264292,
          0.01147406, -0.02281868],
        [ 0.05290215, -0.03380545,  0.06093838, ...,  0.06677757,
          0.00228953,  0.0051606 ],
        ...,
        [ 0.02954119,  0.02109104,  0.0712281 , ...,  0.01830088,
          0.05430136,  0.02652512],
        [ 0.01524809,  0.01126303,  0.05917569, ...,  0.00049231,
          0.02038612, -0.08786622],
        [ 0.08145417,  0.03187617,  0.07208585, ..., -0.02405494,
          0.0391557 ,  0.00609524]],

       [[-0.01036389,  0.02318658,  0.04365764, ...,  0.0020917 ,
          0.00108375,  0.02792811],
        [-0.01303279, -0.00939311,  0.03490455, ...,  0.00327179,
          0.03049978, -0.08557211],
        [ 0.01992651,  0.00969533, -0.02897304, ..., -0.03797258,
          0.0065158 ,  0.02314282],
        ...,
        [-0.00243311,  0.0309361 ,  0.01980856, ...,  

In [ ]:
last_layer = model.get_layer('block5_pool').output ## here i am using "fc2 layer 
x = Flatten(name='flatten')(last_layer)
x = Dense(32, activation='relu', name='fc1')(x)

out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg_model3 = Model(image_input, out)

In [ ]:
look_back=7##here i am giving to 4 words at a time stamp
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=100, 
                    output_dim=150)) # Add an embedding layer which represents each unique token as a vector
model.add(Dropout(0.4))
model.add(LSTM(12,input_shape=(1, look_back), return_sequences=False)) # Add an LSTM layer


model.add(Dense(3, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy',recall])
model.fit(train_x, train_y, epochs=20, validation_split=0.25)